# Building the Neural Network

## Library Imports

In [13]:
#### KERAS IMPORTS ####
from keras import backend as K
K.set_image_dim_ordering('tf')

import keras
from keras.models import Sequential
from keras import layers
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

#### OTHER IMPORTS ####
import cv2
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import joblib

from PIL import ImageGrab
import matplotlib.pyplot as plt

% matplotlib inline

print("BACKEND: ", keras.backend.backend())

BACKEND:  tensorflow


## Functions

In [2]:
def prepare_image(image_path, model='vgg_16'):
    '''
    DESCRIPTION:
        - Preprocess an image to be in the correct size
    INPUT:
        - image_path is the location of the image file
        - model is the type of pretrained CNN
    OUTPUT:
        - resized is the resized image
    '''
    if model == 'vgg_16':
        resized = cv2.resize(cv2.imread(image_path), (224, 224)).astype(np.float32)

        resized[:,:,0] -= 103.939
        resized[:,:,1] -= 116.779
        resized[:,:,2] -= 123.68
        resized = resized.transpose((2,0,1))
        resized = np.expand_dims(resized, axis=0)
    return resized

In [104]:
def resize_img(orig_img, new_dim):
    '''
    DESCRIPTION:
        - resizes the original image.
    INPUT: 
        - orig_img is a numpy array (use cv2.imread() to transform img into numpy array).
        - new_dim is the base number of pixels for the new image.
    OUTPUT:
        - resized is a numpy array of the resized image.
    '''
    r = float(new_dim) / orig_img.shape[1]
    dim = (new_dim, int(orig_img.shape[0] * r))
    resized = cv2.resize(orig_img, dim, interpolation=cv2.INTER_AREA)
    
#     plt.imshow(resized)
#     plt.xticks([])
#     plt.yticks([])
    
    return resized

In [105]:
def rotate_img(orig_img, deg_rot, scale):
    '''
    DESCRIPTION:
        - rotates the original image.
    INPUT: 
        - orig_img is a numpy array (use cv2.imread() to transform img into numpy array).
        - scale (btwn 0 and 1) zooms in on the image. scale (> 1) zooms out on the image. 
        - scale can be used to crop the image based only on the center.
    OUTPUT:
        - rotated_img is a numpy array of the rotated image.
    '''
    (height, width) = orig_img.shape[:2]
    center = (width/2, height/2)
    matrix = cv2.getRotationMatrix2D(center,
                                     angle=deg_rot,
                                     scale=scale)
    rotated_img = cv2.warpAffine(orig_img,
                                 matrix,
                                 (width, height))
#     plt.imshow(rotated_img)
#     plt.xticks([])
#     plt.yticks([])
    
    return rotated_img

In [106]:
def crop_img(orig_img, h1, h2, w1, w2):
    '''
    DESCRIPTION:
        - crops the original image.
    INPUT: 
        - orig_img is a numpy array (use cv2.imread() to transform img into numpy array).
        - h1 and h2 defines height
        - w1 and w2 defines the width
    OUTPUT:
        - cropped_img is a numpy array of the cropped image.
    '''
    cropped_img = orig_img[h1:h2, w1:w2]
    
#     plt.imshow(cropped_img)
#     plt.xticks([])
#     plt.yticks([])

    return cropped_img

In [107]:
def random_aug(image_path):
    '''
    DESCRIPTION:
        - randomly augments the image.
    INPUT: 
        - orig_img is a numpy array (use cv2.imread() to transform img into numpy array).
    OUTPUT:
        - new_img is a numpy array of the augmented image.
    '''
    aug_method = np.random.randint(1,4)
    
    img_arr = cv2.imread(image_path)
#     print("ORIG IMG SHAPE: ", img_arr.shape)
    
    if aug_method == 1:
#         print("RESIZE")
        new_dim = int(img_arr.shape[1] * np.random.uniform(low=0.1, high=0.9))
        new_img_arr = resize_img(img_arr, new_dim)
        
    elif aug_method == 2:
#         print("ROTATE")
        deg = np.random.randint(1, 360)
        scale = np.random.uniform(low=1, high=4)
        new_img_arr = rotate_img(img_arr, deg, scale)
    
    elif aug_method == 3:
#         print("CROP")
        lower_height = np.random.randint(1, img_arr.shape[0])
        lower_width = np.random.randint(1, img_arr.shape[1])
        upper_height = np.random.randint(lower_height, 10000)
        upper_width = np.random.randint(lower_width, 10000)
        
#         print(lower_height, upper_height, lower_width, upper_width)
        new_img_arr = crop_img(img_arr, h1=lower_height, h2=upper_height, w1=lower_width, w2=upper_width)
        
    new_img_path = 'data/aug_images/' + 'aug_' + image_path.replace('data/humansofny/', '')
    
    new_img = cv2.imwrite(new_img_path, new_img_arr)
    if not new_img:
        print("Check image path: ", new_img_path)
    
    return new_img_arr, new_img_path

In [113]:
def get_clean_aug_arrays(tot_count, files):
    '''
    DESCRIPTION:
        - produces a list of numpy arrays for each image in the files list.
    INPUT: 
        - tot_count is the number of files to traverse through.
        - files is the list of image files.
    OUTPUT:
        - X is the list of numpy arrays for the clean images.
        - X_aug is the list of numpy arrays for the augmented images.
    '''
    X = []
    X_aug = []
    
    if tot_count > len(files):
        print("tot_count exceeds the number of files.")
        return False
    
    for i in range(tot_count):
        # Convert the clean image
        X.append(prepare_image(files[i]))

        # Augment then convert the new image
        temp_img_arr, temp_img = random_aug(files[i])
        X_aug.append(prepare_image(temp_img))

    if len(X) == len(X_aug):
        print("Augmenting worked correctly.")
        
    return X, X_aug

## Preprocessing the data

### Get a list of filenames corresponding to the images

In [3]:
files = ['data/humansofny/' + f for f in listdir('data/humansofny/') if isfile(join('data/humansofny/', f))]

### Turn each image into a numpy array

### Create "bad" images by resizing, rotating, and cropping the "good" images

In [114]:
# Full numpy array is ~2.5 GB of data.  Use just 10% of the data to create the pipeline.
tot_count = int(len(files) * 0.1)
X_tenth, X_aug_tenth = get_clean_aug_arrays(tot_count, files)

Augmenting worked correctly.


#### Pickle the X matrix

In [115]:
joblib.dump(X_tenth, 'data/X_tenth.pkl')
joblib.dump(X_aug_tenth, 'data/X_aug_tenth.pkl')
# joblib.dump(X, 'data/X.pkl')

['data/X_aug_tenth.pkl']

#### Load the X matrix

In [11]:
X_tenth = joblib.load('data/X_tenth.pkl')
X_aug_tenth = joblib.load('data/X_aug_tenth.pkl')

# X = joblib.load('data/X.pkl')